<h1 align="center" style="margin: 0; font-size: 36px;">Computational Intelligence for Optimization</h1>
<br>
<h1 align="center" style="margin: 0; font-size: 30px;">Music Festival Lineup Optimization</h1>

<br>

**Group members:**<br>
Bárbara Capitão - 20211532@novaims.unl.pt <br>
Carolina Silvestre - 20211512@novaims.unl.pt <br>
Francisco Pontes -  <br>
Lara Leandro - 20211632@novaims.unl.pt <br>


### Objective
Design the optimal festival lineup by assigning each artist to a specific stage and time slot, optimizing three key criteria:

1. **Prime Slot Popularity**  
   Maximize the total popularity of artists scheduled in the final time slot (prime slot) on each stage.

2. **Genre Diversity**  
   Maximize the diversity of musical genres across stages in each time slot. For every time slot, compute the average number of unique genres, normalized by the number of stages.

3. **Conflict Penalty**  
   Minimize fan base conflicts by avoiding scheduling artists with overlapping fan bases at the same time on different stages. This penalty is calculated using a provided pairwise conflict matrix.

---

### Constraints
- Each artist must be assigned to **exactly one stage and one time slot**.
- All stages must have **the same number of time slots**.
- All time slots occur **simultaneously** across all stages.
- No artist can be scheduled more than once or left unassigned.

---

### Representation
- The lineup is modeled as a 2D matrix:  
  `lineup[stage][slot] = artist_id`
- There are **5 stages and 7 time slots**, for a total of **35 artists**.

---



## Imports and Setup

In [1]:
import numpy as np
import pandas as pd
import random
from collections import defaultdict

random.seed(42)
np.random.seed(42)

## Load Data

In [2]:
artists = pd.read_csv('artists(in).csv')

In [3]:
conflict_matrix = pd.read_csv('conflicts(in).csv', index_col=0)

In [4]:
artists.rename(columns={'Unnamed: 0': 'artist_id'}, inplace=True)
artists.set_index('artist_id', inplace=True)

In [5]:
artists 

,name,genre,popularity
artist_id,,,
0,Midnight Echo,Rock,75
1,Solar Flare,Electronic,78
2,Velvet Pulse,Jazz,35
3,Neon Reverie,Electronic,100
4,The Silver Owls,Classical,85
5,Echo Chamber,Electronic,98
6,Aurora Skies,Pop,75
7,Static Mirage,Rock,94
8,Crimson Harmony,Classical,20


In [6]:
conflict_matrix

,Midnight Echo,Solar Flare,Velvet Pulse,Neon Reverie,The Silver Owls,Echo Chamber,Aurora Skies,Static Mirage,Crimson Harmony,Deep Resonance,...,Rhythm Alchemy,Cloud Nine Collective,Hypnotic Echoes,The Polyrhythm Syndicate,Harmonic Dissonance,Turbo Vortex,The Jazz Nomads,The Bassline Architects,Cosmic Frequency,Parallel Dimension
Midnight Echo,0.00,0.00,0.00,0.20,0.50,0.00,0.80,1.00,0.20,0.50,...,0.20,0.80,1.00,1.00,0.65,1.00,0.40,0.40,1.00,0.20
Solar Flare,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.65,...,0.20,0.65,0.00,0.65,0.40,0.40,0.40,0.00,0.00,1.00
Velvet Pulse,0.00,0.00,0.00,1.00,0.50,0.00,0.65,1.00,0.50,1.00,...,1.00,1.00,0.40,1.00,0.70,0.00,1.00,0.15,1.00,0.40
Neon Reverie,0.20,1.00,1.00,0.00,0.20,0.90,0.20,1.00,0.00,1.00,...,0.00,0.00,0.00,0.20,0.65,0.65,0.20,0.00,0.20,1.00
The Silver Owls,0.50,0.00,0.50,0.20,0.00,1.00,0.00,1.00,0.90,0.00,...,1.00,0.65,0.00,0.20,0.90,0.20,0.20,0.40,1.00,0.00
Echo Chamber,0.00,1.00,0.00,0.90,1.00,0.00,0.50,1.00,0.80,0.65,...,0.00,0.20,0.00,0.50,0.65,0.15,0.50,0.40,1.00,1.00
Aurora Skies,0.80,0.00,0.65,0.20,0.00,0.50,0.00,1.00,1.00,0.40,...,0.00,1.00,1.00,0.20,0.80,0.65,0.20,1.00,0.20,0.50
Static Mirage,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.20,0.20,...,1.00,0.80,0.90,0.65,0.00,1.00,0.40,0.50,1.00,0.20
Crimson Harmony,0.20,0.00,0.50,0.00,0.90,0.80,1.00,0.20,0.00,0.00,...,0.00,0.40,0.65,0.00,0.90,0.15,1.00,1.00,0.65,0.15
Deep Resonance,0.50,0.65,1.00,1.00,0.00,0.65,0.40,0.20,0.00,0.00,...,0.90,0.50,0.20,1.00,0.70,1.00,1.00,0.50,0.00,0.20


## Generating Random Solution

In [7]:
num_artists = len(artists)
num_conflict_rows, num_conflict_cols = conflict_matrix.shape

# Ensure the number of artists matches the expected total
expected_artists = 5 * 7  # 5 stages x 7 slots
consistency_check = (num_artists == expected_artists and 
                     num_conflict_rows == expected_artists and 
                     num_conflict_cols == expected_artists)

# Ensure conflict matrix aligns with artist names
artist_names_match = list(artists['name']) == list(conflict_matrix.columns)

consistency_check, artist_names_match

(True, True)

In [8]:
artist_indices = list(range(35))
random.shuffle(artist_indices)
lineup = np.array(artist_indices).reshape((5, 7))  

# Helper functions
def get_prime_slot_popularity(lineup):
    prime_artists = [lineup[stage][-1] for stage in range(5)]
    total_popularity = sum(artists.iloc[artist]['popularity'] for artist in prime_artists)
    max_possible = 5 * 100  # Max popularity (100) in each of 5 prime slots
    return total_popularity / max_possible

def get_genre_diversity_score(lineup):
    genres = artists['genre'].unique()
    max_unique = len(genres)
    total_score = 0

    for slot in range(7):
        genres_in_slot = set()
        for stage in range(5):
            artist_index = lineup[stage][slot]
            genres_in_slot.add(artists.iloc[artist_index]['genre'])
        total_score += len(genres_in_slot) / max_unique
    
    return total_score / 7  # Average over all slots

def get_conflict_penalty_score(lineup):
    worst_conflict = 0
    actual_conflict = 0

    for slot in range(7):
        artists_in_slot = [lineup[stage][slot] for stage in range(5)]
        for i in range(5):
            for j in range(i + 1, 5):
                a_i = artists.iloc[artists_in_slot[i]]['name']
                a_j = artists.iloc[artists_in_slot[j]]['name']
                conflict = conflict_matrix.at[a_i, a_j]
                actual_conflict += conflict
                worst_conflict += 1  # worst case: all conflicts = 1

    normalized_conflict = actual_conflict / worst_conflict if worst_conflict else 0
    return 1 - normalized_conflict  # because conflict is a penalty
    
lineup_tuple = (
    (3, 17, 23, 34, 31, 16,  9),
    (25,  2, 24, 10, 30, 15, 26),
    (22, 14, 18,  6,  7, 28,  5),
    (29, 19, 11, 32, 27, 13, 21),
    (33, 12,  4,  0,  8,  1, 20)
)

# Compute scores
prime_score = get_prime_slot_popularity(lineup_tuple)
genre_score = get_genre_diversity_score(lineup_tuple)
conflict_score = get_conflict_penalty_score(lineup_tuple)

# Final fitness (average of normalized scores)
fitness = (prime_score + genre_score + conflict_score) / 3

lineup_tuple, {
    'Prime Slot Popularity': round(float(prime_score),4),
    'Genre Diversity': round(float(genre_score),4),
    'Conflict Penalty': round(float(conflict_score),4),
    'Final Fitness': round(float(fitness),4)
}


(((3, 17, 23, 34, 31, 16, 9),
  (25, 2, 24, 10, 30, 15, 26),
  (22, 14, 18, 6, 7, 28, 5),
  (29, 19, 11, 32, 27, 13, 21),
  (33, 12, 4, 0, 8, 1, 20)),
 {'Prime Slot Popularity': 0.936,
  'Genre Diversity': 0.7381,
  'Conflict Penalty': 0.6193,
  'Final Fitness': 0.7645})

### Artist Line up Class

In [43]:
class ArtistLineupSolution:
    def __init__(self, lineup=None):
        if lineup is not None:
            self.lineup = lineup
        else:
            artist_indices = list(range(35))
            random.shuffle(artist_indices)
            self.lineup = np.array(artist_indices).reshape((5, 7))

    def fitness(self):
        def get_prime_slot_popularity():
            prime_artists = [self.lineup[stage, -1] for stage in range(5)]
            total_popularity = sum(artists.iloc[artist]['popularity'] for artist in prime_artists)
            return total_popularity / (5 * 100)

        def get_genre_diversity_score():
            genres = artists['genre'].unique()
            total_score = 0
            for slot in range(7):
                genres_in_slot = set()
                for stage in range(5):
                    artist_index = self.lineup[stage][slot]
                    genres_in_slot.add(artists.iloc[artist_index]['genre'])
                total_score += len(genres_in_slot) / len(genres)
            return total_score / 7

        def get_conflict_penalty_score():
            actual_conflict = 0
            worst_conflict = 0
            for slot in range(7):
                artists_in_slot = [self.lineup[stage][slot] for stage in range(5)]
                for i in range(5):
                    for j in range(i + 1, 5):
                        a_i = artists.iloc[artists_in_slot[i]]['name']
                        a_j = artists.iloc[artists_in_slot[j]]['name']
                        actual_conflict += conflict_matrix.at[a_i, a_j]
                        worst_conflict += 1
            return 1 - (actual_conflict / worst_conflict if worst_conflict else 0)

        return (get_prime_slot_popularity() + get_genre_diversity_score() + get_conflict_penalty_score()) / 3


### Mutation Functions

In [27]:
# Mutation functions
def swap_mutation(individual):
    lineup = individual.lineup
    a, b = random.sample(range(35), 2)
    pos_a = np.argwhere(lineup == a)[0]
    pos_b = np.argwhere(lineup == b)[0]
    lineup[pos_a[0], pos_a[1]], lineup[pos_b[0], pos_b[1]] = lineup[pos_b[0], pos_b[1]], lineup[pos_a[0], pos_a[1]]

def scramble_mutation(individual):
    lineup = individual.lineup.flatten()
    idx = sorted(random.sample(range(35), 5))
    scrambled = [lineup[i] for i in idx]
    random.shuffle(scrambled)
    for i, val in zip(idx, scrambled):
        lineup[i] = val
    individual.lineup = lineup.reshape((5, 7))

def inversion_mutation(individual):
    lineup = individual.lineup.flatten()
    a, b = sorted(random.sample(range(35), 2))
    lineup[a:b+1] = lineup[a:b+1][::-1]
    individual.lineup = lineup.reshape((5, 7))

In [28]:
def displacement_mutation(individual):
    lineup = individual.lineup.flatten().tolist()
    a, b = sorted(random.sample(range(len(lineup)), 2))
    segment = lineup[a:b+1]
    del lineup[a:b+1]
    insert_at = random.randint(0, len(lineup))
    lineup = lineup[:insert_at] + segment + lineup[insert_at:]
    individual.lineup = np.array(lineup).reshape(5, 7)


In [29]:
def heuristic_mutation(individual):
    worst_slot = -1
    worst_score = -1
    for slot in range(7):
        conflict = 0
        for i in range(5):
            for j in range(i + 1, 5):
                a = individual.lineup[i, slot]
                b = individual.lineup[j, slot]
                conflict += conflict_matrix.iat[a, b]
        if conflict > worst_score:
            worst_score = conflict
            worst_slot = slot
    if worst_slot != -1:
        # Swap first two in worst slot
        individual.lineup[0, worst_slot], individual.lineup[1, worst_slot] = individual.lineup[1, worst_slot], individual.lineup[0, worst_slot]


### Crossover Functions

In [30]:
def cycle_crossover_wrapper(p1, p2):
    parent1 = p1.lineup.flatten()
    parent2 = p2.lineup.flatten()
    size = len(parent1)
    child1 = [-1] * size
    child2 = [-1] * size
    indices = list(range(size))
    cycle = 0

    while -1 in child1:
        idx = indices[child1.index(-1)]
        val = parent1[idx]
        start = idx
        while True:
            child1[start] = parent1[start] if cycle % 2 == 0 else parent2[start]
            child2[start] = parent2[start] if cycle % 2 == 0 else parent1[start]
            start = parent1.tolist().index(parent2[start])
            if start == idx:
                break
        cycle += 1

    return ArtistLineupSolution(np.array(child1).reshape(5, 7)), ArtistLineupSolution(np.array(child2).reshape(5, 7))

In [31]:
def order_crossover_wrapper(p1, p2):
    parent1 = p1.lineup.flatten().tolist()
    parent2 = p2.lineup.flatten().tolist()
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))

    def order_crossover(p1, p2):
        child = [-1]*size
        child[a:b+1] = p1[a:b+1]
        fill = [item for item in p2 if item not in child]
        idx = 0
        for i in range(size):
            if child[i] == -1:
                child[i] = fill[idx]
                idx += 1
        return child

    child1 = order_crossover(parent1, parent2)
    child2 = order_crossover(parent2, parent1)
    return ArtistLineupSolution(np.array(child1).reshape(5, 7)), ArtistLineupSolution(np.array(child2).reshape(5, 7))


In [32]:
def pmx_crossover_wrapper(p1, p2):
    parent1 = p1.lineup.flatten()
    parent2 = p2.lineup.flatten()
    size = len(parent1)
    cx_point1, cx_point2 = sorted(random.sample(range(size), 2))

    def pmx(parent_a, parent_b):
        child = [-1] * size
        child[cx_point1:cx_point2+1] = parent_a[cx_point1:cx_point2+1]
        for i in range(cx_point1, cx_point2+1):
            if parent_b[i] not in child:
                pos = i
                while child[pos] != -1:
                    pos = parent_b.tolist().index(parent_a[pos])
                child[pos] = parent_b[i]
        for i in range(size):
            if child[i] == -1:
                child[i] = parent_b[i]
        return child

    child1 = pmx(parent1, parent2)
    child2 = pmx(parent2, parent1)
    return ArtistLineupSolution(np.array(child1).reshape(5, 7)), ArtistLineupSolution(np.array(child2).reshape(5, 7))

### Selection Functions

In [33]:
def tournament_selection(population, k=3):
    selected = random.sample(population, k)
    return max(selected, key=lambda x: x.fitness()), max(random.sample(population, k), key=lambda x: x.fitness())

In [34]:
def roulette_selection(population):
    total_fitness = sum(p.fitness() for p in population)
    probs = [p.fitness() / total_fitness for p in population]
    return np.random.choice(population, p=probs), np.random.choice(population, p=probs)

In [35]:
def rank_selection(population):
    sorted_pop = sorted(population, key=lambda x: x.fitness())
    ranks = list(range(1, len(population) + 1))
    total = sum(ranks)
    probs = [rank / total for rank in ranks]
    return np.random.choice(sorted_pop, p=probs), np.random.choice(sorted_pop, p=probs)


### Genetic Algorithm

In [36]:
class GeneticAlgorithm:
    def __init__(self, selection_functions, mutation_functions, crossover_functions,
                 population_size=50, generations=100):
        self.selection_functions = selection_functions
        self.mutation_functions = mutation_functions
        self.crossover_functions = crossover_functions
        self.population_size = population_size
        self.generations = generations

    def run(self):
        population = [ArtistLineupSolution() for _ in range(self.population_size)]
        best_solution = max(population, key=lambda x: x.fitness())

        for gen in range(self.generations):
            new_population = []
            for _ in range(self.population_size // 2):
                selection = random.choice(self.selection_functions)
                parent1, parent2 = selection(population)
                crossover = random.choice(self.crossover_functions)
                child1, child2 = crossover(parent1, parent2)

                random.choice(self.mutation_functions)(child1)
                random.choice(self.mutation_functions)(child2)

                new_population.extend([child1, child2])

            population = new_population
            best_in_gen = max(population, key=lambda x: x.fitness())
            if best_in_gen.fitness() > best_solution.fitness():
                best_solution = best_in_gen

            print(f"Gen {gen + 1}: Best Fitness = {round(best_solution.fitness(), 4)}")

        return best_solution

In [37]:
if __name__ == "__main__":
    ga = GeneticAlgorithm(
        selection_functions=[tournament_selection, roulette_selection],
        mutation_functions=[swap_mutation, scramble_mutation, inversion_mutation],
        crossover_functions=[cycle_crossover_wrapper, pmx_crossover_wrapper],
        population_size=50,
        generations=100
    )

    best = ga.run()
    print("Best lineup:")
    print(best.lineup)
    print("Fitness Score:", round(best.fitness(), 4))


Gen 1: Best Fitness = 0.712


KeyboardInterrupt: 

In [38]:
if __name__ == "__main__":
    ga = GeneticAlgorithm(
        selection_functions=[tournament_selection, roulette_selection, rank_selection],
        mutation_functions=[swap_mutation, scramble_mutation, inversion_mutation, displacement_mutation, heuristic_mutation],
        crossover_functions=[cycle_crossover_wrapper, pmx_crossover_wrapper, order_crossover_wrapper],
        population_size=50,
        generations=100
    )

    best = ga.run()
    print("Best lineup:")
    print(best.lineup)
    print("Fitness Score:", round(best.fitness(), 4))


Gen 1: Best Fitness = 0.7251
Gen 2: Best Fitness = 0.7251
Gen 3: Best Fitness = 0.7251
Gen 4: Best Fitness = 0.7251
Gen 5: Best Fitness = 0.7251
Gen 6: Best Fitness = 0.7251
Gen 7: Best Fitness = 0.7339
Gen 8: Best Fitness = 0.7339
Gen 9: Best Fitness = 0.7339
Gen 10: Best Fitness = 0.7339
Gen 11: Best Fitness = 0.7339
Gen 12: Best Fitness = 0.7339
Gen 13: Best Fitness = 0.7456
Gen 14: Best Fitness = 0.7456
Gen 15: Best Fitness = 0.7456
Gen 16: Best Fitness = 0.7456
Gen 17: Best Fitness = 0.7456
Gen 18: Best Fitness = 0.7456
Gen 19: Best Fitness = 0.7456
Gen 20: Best Fitness = 0.7456
Gen 21: Best Fitness = 0.7456
Gen 22: Best Fitness = 0.7456
Gen 23: Best Fitness = 0.7456
Gen 24: Best Fitness = 0.7456
Gen 25: Best Fitness = 0.7456
Gen 26: Best Fitness = 0.7456
Gen 27: Best Fitness = 0.7456
Gen 28: Best Fitness = 0.7456
Gen 29: Best Fitness = 0.7456
Gen 30: Best Fitness = 0.7456
Gen 31: Best Fitness = 0.7463
Gen 32: Best Fitness = 0.7463
Gen 33: Best Fitness = 0.7463
Gen 34: Best Fitnes

In [ ]:
'''
Best lineup:
[[ 3 17 23 34 31 16  9]
 [25  2 24 10 30 15 26]
 [22 14 18  6  7 28  5]
 [29 19 11 32 27 13 21]
 [33 12  4  0  8  1 20]]
Fitness Score: 0.7645
'''

In [49]:
import numpy as np
import pandas as pd
import random
import math

# Assume these are already defined:
# artists (DataFrame with columns: name, popularity, genre)
# conflict_matrix (DataFrame: index and columns are artist names with conflict values)

class ArtistLineupSolution:
    def __init__(self, lineup=None):
        if lineup is not None:
            self.lineup = lineup
        else:
            artist_indices = list(range(35))
            random.shuffle(artist_indices)
            self.lineup = np.array(artist_indices).reshape((5, 7))

    def fitness(self):
        def get_prime_slot_popularity():
            prime_artists = [self.lineup[stage, -1] for stage in range(5)]
            total_popularity = sum(artists.iloc[artist]['popularity'] for artist in prime_artists)
            return total_popularity / (5 * 100)

        def get_genre_diversity_score():
            genres = artists['genre'].unique()
            total_score = 0
            for slot in range(7):
                genres_in_slot = set()
                for stage in range(5):
                    artist_index = self.lineup[stage][slot]
                    genres_in_slot.add(artists.iloc[artist_index]['genre'])
                total_score += len(genres_in_slot) / len(genres)
            return total_score / 7

        def get_conflict_penalty_score():
            actual_conflict = 0
            worst_conflict = 0
            for slot in range(7):
                artists_in_slot = [self.lineup[stage][slot] for stage in range(5)]
                for i in range(5):
                    for j in range(i + 1, 5):
                        a_i = artists.iloc[artists_in_slot[i]]['name']
                        a_j = artists.iloc[artists_in_slot[j]]['name']
                        actual_conflict += conflict_matrix.at[a_i, a_j]
                        worst_conflict += 1
            return 1 - (actual_conflict / worst_conflict if worst_conflict else 0)

        return (get_prime_slot_popularity() + get_genre_diversity_score() + get_conflict_penalty_score()) / 3

    def mutate(self):
        a, b = random.sample(range(35), 2)
        pos_a = np.argwhere(self.lineup == a)[0]
        pos_b = np.argwhere(self.lineup == b)[0]
        self.lineup[pos_a[0], pos_a[1]], self.lineup[pos_b[0], pos_b[1]] = self.lineup[pos_b[0], pos_b[1]], self.lineup[pos_a[0], pos_a[1]]

    def copy(self):
        return ArtistLineupSolution(np.copy(self.lineup))

    
    def __str__(self):
        return f"Lineup:\n{self.lineup}\nFitness: {self.fitness():.4f}"

def hill_climbing(max_iterations=1000):
    current = ArtistLineupSolution()
    current_fitness = current.fitness()
    for _ in range(max_iterations):
        neighbor = current.copy()
        neighbor.mutate()
        neighbor_fitness = neighbor.fitness()
        if neighbor_fitness > current_fitness:
            current, current_fitness = neighbor, neighbor_fitness
    return current

def simulated_annealing(initial_temp=100, cooling_rate=0.99, max_iterations=1000):
    current = ArtistLineupSolution()
    current_fitness = current.fitness()
    temp = initial_temp
    for _ in range(max_iterations):
        neighbor = current.copy()
        neighbor.mutate()
        neighbor_fitness = neighbor.fitness()
        delta = neighbor_fitness - current_fitness
        if delta > 0 or random.random() < math.exp(delta / temp):
            current, current_fitness = neighbor, neighbor_fitness
        temp *= cooling_rate
    return current


In [51]:
solution = hill_climbing(max_iterations=1000)
print(solution)

Lineup:
[[29 34 18 25 17 24  9]
 [23 27 31  6  5 26 14]
 [ 7 28 12 16  2 20  3]
 [10 11 15  1 30  8 13]
 [32  4 21 33  0 19 22]]
Fitness: 0.8243


In [ ]:
'''
Lineup:
[[29 34 18 25 17 24  9]
 [23 27 31  6  5 26 14]
 [ 7 28 12 16  2 20  3]
 [10 11 15  1 30  8 13]
 [32  4 21 33  0 19 22]]
Fitness: 0.8243
'''

In [52]:
solution = simulated_annealing(initial_temp=100, cooling_rate=0.99, max_iterations=1000)
print(solution)

Lineup:
[[ 9 18 19  2 34 20 17]
 [12  6  5 21 30 26 23]
 [ 1 25 33  0 10  4 13]
 [31 22 24 11  7 32 27]
 [15  3  8 29 16 28 14]]
Fitness: 0.7410
